In [ ]:
import pandas as pd
import ewtpy
import pywt
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import initializers

In [ ]:
df = pd.read_csv('data_day.csv')
print(df.to_string())
data = df.to_numpy()
temp = df['temp'].to_numpy()
temp_max = df['tempmax'].to_numpy()
temp_min = df['tempmin'].to_numpy()
humidity = df['humidity'].to_numpy()
windspeed = df['windspeed'].to_numpy()
solar_radiation = df['solarradiation'].to_numpy()
precipitation = df['precip'].to_numpy()
dew = df['dew'].to_numpy()
precipcover = df['precipcover'].to_numpy()
winddir = df['winddir'].to_numpy()
solarenergy = df['solarenergy'].to_numpy()
training_set = np.array([temp_max, temp_min ,solar_radiation, precipitation, windspeed])
NUM = 0
plt.plot(humidity)

In [ ]:

training_set_tp = training_set.transpose()

In [ ]:
X = training_set_tp
y = np.array([humidity]).transpose()
print(X.shape)
print(y.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=False
)
save = y_test
y_train = np.array([y_train[i + NUM] for i in range(len(y_train) - NUM)])
y_test = np.array([y_test[i + NUM] for i in range(len(y_test) - NUM)])

In [ ]:
# model = Sequential(
#     [
#         tf.keras.Input(shape=(5,)),
#         tf.keras.layers.Dense(units = 25, activation = "relu"),
#         tf.keras.layers.Dense(units = 15, activation = "relu"),
#         tf.keras.layers.Dense(units = 5, activation = "relu"),
#         tf.keras.layers.Dense(units = 1, activation = "linear")
#         ### END CODE HERE ### 
#     ], name = "mlpnn_model" 
# ) 
# model.compile(
#     loss=tf.keras.losses.MeanSquaredError(),
#     optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
# )
# if (NUM != 0):
#     X_train = X_train[:-NUM]
# else:
#     X_train = X_train[:]
# print(X_train.shape)
# # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
# hist_mlpnn = model.fit(
#     X_train,y_train,
#     epochs=1000,  
#     # callbacks = [callback] 
# ) 

In [ ]:
model = Sequential(
    [
        tf.keras.Input(shape=(5,)),
        tf.keras.layers.Dense(units = 1000, activation = "relu",kernel_initializer=initializers.RandomNormal(stddev=0.01),
    bias_initializer=initializers.Zeros()),
        tf.keras.layers.Dense(units = 1, activation = "linear")
        ### END CODE HERE ### 
    ], name = "elm_model" 
) 
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
)
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
if (NUM != 0):
    X_train = X_train[:-NUM]
else:
    X_train = X_train[:]
hist_elm = model.fit(
    X_train,y_train,
    epochs=500,   
    # callbacks = [callback]
) 


In [ ]:
yt_predict = model.predict(X_train)
y_predict = model.predict(X_test)
if (NUM != 0):
    y_predict = y_predict[:-NUM]
    # yt_predict = yt_predict[:-NUM]
plt.scatter(yt_predict[:,0],y_train[:,0])
plt.scatter(y_predict[:,0],y_test[:,0])
for i in range(len(y_predict) - NUM):
    print(f"predict: {y_predict[i,0]}, test: {y_test[i, 0]}")

print("TRAIN DETAIL")
print(f"R: {np.corrcoef(yt_predict[:,0],y_train[:,0])}")
print(f"RMSE: {mean_squared_error(y_train[:,0],yt_predict[:,0])**(1/2)}")
print(f"MAE: {mean_absolute_error(y_train[:,0],yt_predict[:,0])}")
# print(f"SCORE: {accuracy_score(y_train[:,0],yt_predict[:,0])}")
mt, bt = np.polyfit(yt_predict[:,0],y_train[:,0], 1)
plt.plot(yt_predict[:,0], mt*yt_predict[:,0]+bt, color = "yellow")
#add linear regression line to scatterplot 
print("TEST DETAIL")
print(f"R: {np.corrcoef(y_predict[:,0],y_test[:,0])}")
print(f"RMSE: {mean_squared_error(y_test[:,0],y_predict[:,0])**(1/2)}")
print(f"MAE: {mean_absolute_error(y_test[:,0],y_predict[:,0])}")
# print(f"SCORE: {accuracy_score(y_test[:,0],y_predict[:,0])}")
m, b = np.polyfit(y_predict[:,0],y_test[:,0], 1)
plt.plot(y_predict[:,0], m*y_predict[:,0]+b, color = "red")


In [ ]:
# plt.plot(history.history['loss'][200:500])
# print(history.history["loss"][400:500])
plt.plot(hist_elm.history['loss'][0:500])